# Phonon computations

This is a quick sketch how to run a simple phonon calculation using DFTK.
First we run an SCF calculation.

In [1]:
using AtomsBuilder
using DFTK
using PseudoPotentialData

pseudopotentials = PseudoFamily("cp2k.nc.sr.lda.v0_1.semicore.gth")
model  = model_DFT(bulk(:Si); pseudopotentials, functionals=LDA())
basis  = PlaneWaveBasis(model; Ecut=10, kgrid=[4, 4, 4])
scfres = self_consistent_field(basis, tol=1e-8);
nothing  # hide

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.916459887894                   -0.69    4.6    113ms
  2   -7.921212924259       -2.32       -1.52    1.0   52.3ms
  3   -7.921405285646       -3.72       -2.49    1.2   54.7ms
  4   -7.921440858056       -4.45       -2.82    2.5   82.6ms
  5   -7.921441619296       -6.12       -3.00    1.1   53.3ms
  6   -7.921442005619       -6.41       -4.52    1.0   52.3ms
  7   -7.921442021710       -7.79       -4.54    3.0   80.6ms
  8   -7.921442022122       -9.39       -5.17    1.0   56.2ms
  9   -7.921442022141      -10.72       -5.35    1.5   57.7ms
 10   -7.921442022143      -11.57       -5.98    1.0   56.4ms
 11   -7.921442022144      -12.14       -6.31    1.6   60.4ms
 12   -7.921442022144      -13.30       -6.65    1.4   59.0ms
 13   -7.921442022144      -14.01       -7.19    1.1   54.7ms
 14   -7.921442022144      -14.57       -8.00    2.0   67.4ms


Next we compute the phonon modes at the q-point `[1/4, 1/4, 1/4]`.

In [2]:
scfres = DFTK.unfold_bz(scfres)
phret_q0 = @time DFTK.phonon_modes(scfres; q=[0.25, 0.25, 0.25]);
nothing  # hide

Iter  Restart  Krydim  log10(res)  avg(CG)  Δtime   Comment
----  -------  ------  ----------  -------  ------  ---------------
                                      72.9   2.66s  Non-interacting
   1        0       1        0.31     52.5   18.0s  
   2        0       2       -0.52     47.0   1.47s  
   3        0       3       -2.40     41.6   1.29s  
   4        0       4       -4.46     29.6   1.03s  
   5        0       5       -6.46     17.4   724ms  
   6        0       6       -8.91      4.0   405ms  
   7        1       1       -7.80     55.9   2.44s  Restart
   8        1       2       -8.77      5.8   464ms  
                                      72.8   1.97s  Final orbitals
Iter  Restart  Krydim  log10(res)  avg(CG)  Δtime   Comment
----  -------  ------  ----------  -------  ------  ---------------
                                      72.9   1.79s  Non-interacting
   1        0       1        0.31     52.5   1.67s  
   2        0       2       -0.52     47.0   1.44s  
   3

These are the final phonon frequencies:

In [3]:
phret_q0.frequencies

6-element Vector{Float64}:
 -0.003498155359859895
 -0.0029774511656817694
 -0.002977451164090287
  0.004302230640614572
  0.004302230642654138
  0.0043532017695473695

> **Preliminary implementation**
>
> Practical phonon computations have only seen rudimentary testing as of now.
> As of now we do not yet recommend relying on this feature for production
> calculations. We appreciate any issues, bug reports or PRs.